In [1]:
# Read padded data from the files 
# Add option to read from padded or to rerun it again.
# =============================================================================
import numpy as np
from numpy import array
from numpy import argmax
import pandas as pd
import os,errno
from time import time
import keras
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.layers import Dense, Input, Dropout,LSTM, Lambda,Bidirectional
from keras.callbacks import ModelCheckpoint,TensorBoard#,TimeDistributed
#from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from keras import backend as K
from itertools import product 
import helpers
from helpers import string_with_tashkeel_vectorizer,string_vectorizer
import arabic
import pyarabic.araby as araby

#from keras.layers.core import

print("Imports Done")
# =============================================================================

Using TensorFlow backend.


Imports Done


In [2]:
np.random.seed(7)
#os.chdir("C:\Users\taha\Desktop\ArabicPoetry-1\Arabic_Poetry_RNN\src")

arabic_alphabet = arabic.alphabet
numberOfUniqueChars = len(arabic_alphabet)
print(numberOfUniqueChars)

36


# Parameters

In [3]:
# =======================Program Parameters====================================
load_weights_flag = 0
#Experiement_Name = 'Experiement_1_WITH_Tashkeel_ASIS'
Experiement_Name = 'Experiement_1_WITH_Tashkeel_ASIS_OldData'
test_size_param=0.5
validation_split_param = 0.01
n_units = 200
input_data_path = "../data/All_Data.csv"
epochs_param = 20
batch_size_param = 50
old_date_flag = 1 # SHIT

In [4]:
#===============================Concatinated Variables ========================

checkpoints_path ="../checkpoints/"+Experiement_Name+"/"
check_points_file_path = checkpoints_path+ "/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
board_log_dir="../logs/"+Experiement_Name+"/"#+.format(time())

try:
    os.makedirs(board_log_dir)
    os.makedirs(checkpoints_path)
except OSError as e:
    if e.errno != errno.EEXIST:
        print("Can't create file for checkpoints or for logs please check ")
        raise
print("Input Parameters Defined and Experiement directory created")

Input Parameters Defined and Experiement directory created


# Loading Data

In [5]:
# =========================Data Loading========================================

sample_arabic_poetry = pd.read_csv(input_data_path, sep = ",")
if (old_date_flag == 1):
    print("working into old data sample ")
    cols = [1,2,4]
    sample_arabic_poetry.drop(sample_arabic_poetry.columns[cols], axis=1,inplace=True)
    sample_arabic_poetry.columns = ['Bayt_Text', 'Category']
    #sample_arabic_poetry['Bayt_Text'] = sample_arabic_poetry['Bayt_Text'].apply(araby.strip_tashkeel).apply(araby.strip_tatweel)
    sample_arabic_poetry['Bayt_Text'] = sample_arabic_poetry['Bayt_Text'].apply(araby.strip_tatweel)

    max_Bayt_length =  sample_arabic_poetry.Bayt_Text.map(len).max()
    print('max_bayt = ' + str(max_Bayt_length))

    # Encoding data
    Bayt_Text_Encoded = sample_arabic_poetry['Bayt_Text'].apply(string_with_tashkeel_vectorizer)
    Bayt_Text_Encoded = np.stack(Bayt_Text_Encoded , axis=0)
    
    
    print('data matrix dimension = ' + str(Bayt_Text_Encoded.shape))
    print('element of data matrix = ' + str(Bayt_Text_Encoded[1].shape))
    print("Input Data Bayt_Text encoded done.")

else:
    print("working on new data sample")
    cols = [0,1,2,3,4,6,7]
    sample_arabic_poetry.drop(sample_arabic_poetry.columns[cols], axis=1,inplace=True)
    sample_arabic_poetry.columns = ['Category', 'Bayt_Text']
    Bayt_Text_Encoded = sample_arabic_poetry['Bayt_Text'].apply(string_with_tashkeel_vectorizer)
    print("Input Data Bayt_Text encoded done.")

    
#sample_arabic_poetry['Bayt_Text'] = sample_arabic_poetry['Bayt_Text'].apply(araby.strip_tashkeel).apply(araby.strip_tatweel)
max_Bayt_length =  sample_arabic_poetry.Bayt_Text.map(len).max()
print("Max Length = " + str(max_Bayt_length))

print("Input Data Read done.")

working into old data sample 
max_bayt = 111
data matrix dimension = (227615, 111, 8)
element of data matrix = (111, 8)
Input Data Bayt_Text encoded done.
Max Length = 111
Input Data Read done.


In [6]:
print(sample_arabic_poetry.columns)
sample_arabic_poetry

Index(['Bayt_Text', 'Category'], dtype='object')


,Bayt_Text,Category
0,حاشا اهتمامك من إسداء مكرمة وأن يرى لك بين الن...,البسيط
1,دع مقصراً ليس بدَّاعاً ولا عجبٌ للأريحية والإح...,البسيط
2,أَيَّتُها النَفسُ لا تُهالي شَرخي قَد مَرَّ وَ...,البسيط
3,لَم يَبقَ إِلّا شَفاً يَسيرٌ قَرَّبَ مِن مَورِ...,البسيط
4,وَاِبتَهَلَ الدَهرُ في أَذاتي وَكانَ في الباطِ...,البسيط
5,وَأُمُّ دَفرٍ فَتاةُ سوءٍ تَخبُؤُني في ثَرىً م...,البسيط
6,مُرسِلَةً غارَةً بِخَيلٍ قَد غَنِيَت عَن هَبٍ ...,البسيط
7,وَجَدتُ حُبّي لَها قَديماً وَقَد تَبَيَّنتُ مَ...,البسيط
8,من سَرَّهُ كسبُ مالٍ لا نفاذَ له وراحةٌ قُرِنت...,البسيط
9,فلا يَمُدَّنَّ عينيه إلى سبب سوى القَناعة من م...,البسيط


# One-Hot encoding for classes

In [7]:
# =============================================================================
#one hot encoding for classes
# =============================================================================
Bayt_Bahr = sample_arabic_poetry['Category']
numbber_of_bohor = Bayt_Bahr.unique().size

# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(Bayt_Bahr)

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
Bayt_Bahr_encoded = onehot_encoder.fit_transform(integer_encoded)

# invert first example
# class for the observation(1)
inverted = label_encoder.inverse_transform([argmax(Bayt_Bahr_encoded[1, :])])
print( 'Classes dimentsion = ' + str(Bayt_Bahr_encoded.shape))
print(inverted)

print("Input Data Category encoded done.")
# =============================================================================

Classes dimentsion = (227615, 11)
['البسيط']
Input Data Category encoded done.


# Splitting data into train and test

In [8]:
X_train, X_test, y_train, y_test = train_test_split( Bayt_Text_Encoded, #bayts
                                                     Bayt_Bahr_encoded, #classes
                                                     test_size = test_size_param, 
                                                     random_state = 0)
#default padding need to check the paramters details
print("Input Train/Test Split done.")
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

Input Train/Test Split done.
(113807, 111, 8)
(113807, 11)
(113808, 111, 8)
(113808, 11)


In [9]:
# =================================Padding=====================================

#X_train_padded = sequence.pad_sequences(X_train, maxlen=max_Bayt_length)
#X_test_padded = sequence.pad_sequences(X_test, maxlen=max_Bayt_length)

X_train_padded = X_train
X_test_padded = X_test

print(X_train_padded.shape)
print(X_train_padded[1].shape)
print(X_train_padded[2].shape)
print(X_train_padded[3].shape)
print(X_test_padded.shape)

print("Padding done.")
# =============================================================================

(113807, 111, 8)
(111, 8)
(111, 8)
(111, 8)
(113808, 111, 8)
Padding done.


# Building Model

In [33]:
# =========================With LSTM Layers ================================
# create model
K.set_learning_phase(1) #set learning phase

n_units = 300
model = Sequential()


# Adding the input layer and the LSTM layer
if (old_date_flag == 1):
    #model.add(LSTM(units = n_units, input_shape=(max_Bayt_length, numberOfUniqueChars), return_sequences=True))
    model.add(LSTM(units = n_units, input_shape=(max_Bayt_length, 8), return_sequences=True))

else:
    model.add(LSTM(units = n_units, input_shape=(max_Bayt_length, 8), return_sequences=True))
    
model.add(Dropout(0.1,seed=7)) 

model.add(LSTM(n_units, return_sequences=True))
model.add(Dropout(0.1,seed=7)) 

model.add(LSTM(n_units, return_sequences=True))
model.add(Dropout(0.1,seed=7)) 

model.add(LSTM(n_units, return_sequences=True))
model.add(Dropout(0.1,seed=7)) 

model.add(LSTM(n_units))
model.add(Dropout(0.1,seed=7)) 



# Adding the output layer
model.add(Dense(units = numbber_of_bohor,activation = 'softmax'))

# load weights
# is set to zero
if(load_weights_flag == 1):
    try:
        #List all avialble checkpoints into the directory
        checkpoints_path_list = os.listdir(checkpoints_path)
        all_checkpoints_list = [os.path.join(checkpoints_path,i) for i in checkpoints_path_list]
        #Get the last inserted weight into the checkpoint_path
        all_checkpoints_list_sorted = sorted(all_checkpoints_list, key=os.path.getmtime)
        print (" max_weight_checkpoints")
        print(all_checkpoints_list_sorted[-1])
        max_weight_checkpoints =  all_checkpoints_list_sorted[-1]
        # load weights
        model.load_weights(max_weight_checkpoints)
    except IOError:
        print('An error occured trying to read the file.')
    except:
        print("No wieghts avialable \n check the paths")


        
        
# Compiling the RNN
model.compile(optimizer = 'adam', 
              loss='categorical_crossentropy',
              metrics = ['accuracy'])
print("Model Defined and compliled.")
print(model.summary())






checkpoint = ModelCheckpoint(check_points_file_path, 
                             monitor='val_acc', 
                             verbose=1,
                             save_best_only=True, 
                             mode='max')
print("Model checkpoint defined to track val_acc")
tensorboard  = keras.callbacks.TensorBoard(log_dir=board_log_dir , 
                                           histogram_freq=0, 
                                           batch_size=batch_size_param, 
                                           write_graph=True, 
                                           write_grads=True, 
                                           write_images=True, 
                                           embeddings_freq=0, 
                                           embeddings_layer_names=None, 
                                           embeddings_metadata=None)
print("Model tensorboard defined")
callbacks_list = [checkpoint,tensorboard]

Model Defined and compliled.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_51 (LSTM)               (None, 111, 300)          370800    
_________________________________________________________________
dropout_51 (Dropout)         (None, 111, 300)          0         
_________________________________________________________________
lstm_52 (LSTM)               (None, 111, 300)          721200    
_________________________________________________________________
dropout_52 (Dropout)         (None, 111, 300)          0         
_________________________________________________________________
lstm_53 (LSTM)               (None, 111, 300)          721200    
_________________________________________________________________
dropout_53 (Dropout)         (None, 111, 300)          0         
_________________________________________________________________
lstm_54 (LSTM)               (None, 111, 300)  

# Train :)

In [ ]:
print("Model Training and validation started")
# Fitting the RNN to the Training set
hist = model.fit(X_train_padded, 
                 y_train, 
                 validation_split = validation_split_param, 
                 epochs=epochs_param, 
                 batch_size=batch_size_param, 
                 callbacks=callbacks_list,
                 verbose=1)

Model Training and validation started
Train on 112668 samples, validate on 1139 samples


In [ ]:
print("Model Training and validation finished")
print(history.losses)

# Final evaluation of the model
scores = model.evaluate(X_test_padded, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# ===========================Ploting===========================================
# list all data in history
print(hist.history.keys())
# summarize history for accuracy
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()